# Minimal Ride Hailing Example

### Configuration

Restart your Kernel after installing these packages

In [1]:
!pip install protobuf gcsfs feast -U -q --user

### Basic Imports and Feast Client initialization

In [2]:
import os
from pprint import pprint
pprint({key: value for key, value in os.environ.items() if key.startswith("FEAST_")})
import os
staging_bucket = 'gs://my-feast-playground-s-11-4744ea70/'
from feast import Client, Feature, Entity, ValueType, FeatureTable
from feast.data_source import FileSource, KafkaSource
from feast.data_format import ParquetFormat, AvroFormat
#client = Client()

client = Client(
    core_url="feast-release-feast-core:6565",
    serving_url="feast-release-feast-serving:6566",
    spark_launcher="k8s",
    spark_staging_location=staging_bucket,
    spark_k8s_namespace="default",
    executor_instances=2,
    redis_host="feast-release-redis-headless",
    historical_feature_output_location=f"{staging_bucket}historical",
)
client.set_project("default")

{'FEAST_CORE_URL': 'feast-release-feast-core:6565',
 'FEAST_HISTORICAL_FEATURE_OUTPUT_LOCATION': 'file:///home/jovyan/historical_feature_output',
 'FEAST_HISTORICAL_SERVING_URL': 'feast-release-feast-batch-serving:6566',
 'FEAST_REDIS_HOST': 'feast-release-redis-master',
 'FEAST_RELEASE_FEAST_CORE_PORT': 'tcp://10.79.242.197:80',
 'FEAST_RELEASE_FEAST_CORE_PORT_6565_TCP': 'tcp://10.79.242.197:6565',
 'FEAST_RELEASE_FEAST_CORE_PORT_6565_TCP_ADDR': '10.79.242.197',
 'FEAST_RELEASE_FEAST_CORE_PORT_6565_TCP_PORT': '6565',
 'FEAST_RELEASE_FEAST_CORE_PORT_6565_TCP_PROTO': 'tcp',
 'FEAST_RELEASE_FEAST_CORE_PORT_80_TCP': 'tcp://10.79.242.197:80',
 'FEAST_RELEASE_FEAST_CORE_PORT_80_TCP_ADDR': '10.79.242.197',
 'FEAST_RELEASE_FEAST_CORE_PORT_80_TCP_PORT': '80',
 'FEAST_RELEASE_FEAST_CORE_PORT_80_TCP_PROTO': 'tcp',
 'FEAST_RELEASE_FEAST_CORE_SERVICE_HOST': '10.79.242.197',
 'FEAST_RELEASE_FEAST_CORE_SERVICE_PORT': '80',
 'FEAST_RELEASE_FEAST_CORE_SERVICE_PORT_GRPC': '6565',
 'FEAST_RELEASE_FEAST_

### Declare Features and Entities

In [3]:
driver_id = Entity(name="driver_id", description="Driver identifier", value_type=ValueType.INT64)

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
# Daily updated features 
acc_rate = Feature("acc_rate", ValueType.FLOAT)
conv_rate = Feature("conv_rate", ValueType.FLOAT)
avg_daily_trips = Feature("avg_daily_trips", ValueType.INT32)

# Real-time updated features
trips_today = Feature("trips_today", ValueType.INT32)
import time
time.sleep(5)

In [5]:
# Offline data will be stored in this location
demo_data_location = "gs://my-feast-playground-s-11-4744ea70/test_data/feb28_11_10pm"
print(demo_data_location)

gs://my-feast-playground-s-11-4744ea70/test_data/feb28_11_10pm


In [6]:
driver_statistics_source_uri = os.path.join(demo_data_location, "driver_statistics")

driver_statistics = FeatureTable(
    name = "driver_statistics",
    entities = ["driver_id"],
    features = [
        acc_rate,
        conv_rate,
        avg_daily_trips
    ],
    batch_source=FileSource(
        event_timestamp_column="datetime",
        created_timestamp_column="created",
        file_format=ParquetFormat(),
        file_url=driver_statistics_source_uri,
        date_partition_column="date"
    )
)

In [7]:
driver_trips_source_uri = os.path.join(demo_data_location, "driver_trips")


driver_trips = FeatureTable(
    name = "driver_trips",
    entities = ["driver_id"],
    features = [
        trips_today
    ],
    batch_source=FileSource(
        event_timestamp_column="datetime",
        created_timestamp_column="created",
        file_format=ParquetFormat(),
        file_url=driver_trips_source_uri,
        date_partition_column="date"
    )
)

### Registering entities and feature tables in Feast Core

In [8]:
client.apply(driver_id)
client.apply(driver_statistics)
client.apply(driver_trips)

In [9]:
print(client.get_feature_table("driver_statistics").to_yaml())
print(client.get_feature_table("driver_trips").to_yaml())

spec:
  name: driver_statistics
  entities:
  - driver_id
  features:
  - name: avg_daily_trips
    valueType: INT32
  - name: acc_rate
    valueType: FLOAT
  - name: conv_rate
    valueType: FLOAT
  batchSource:
    type: BATCH_FILE
    eventTimestampColumn: datetime
    datePartitionColumn: date
    createdTimestampColumn: created
    fileOptions:
      fileFormat:
        parquetFormat: {}
      fileUrl: gs://my-feast-playground-s-11-4744ea70/test_data/feb28_11_10pm/driver_statistics
meta:
  createdTimestamp: '2021-03-12T06:06:14Z'

spec:
  name: driver_trips
  entities:
  - driver_id
  features:
  - name: trips_today
    valueType: INT32
  batchSource:
    type: BATCH_FILE
    eventTimestampColumn: datetime
    datePartitionColumn: date
    createdTimestampColumn: created
    fileOptions:
      fileFormat:
        parquetFormat: {}
      fileUrl: gs://my-feast-playground-s-11-4744ea70/test_data/feb28_11_10pm/driver_trips
meta:
  createdTimestamp: '2021-03-12T06:06:15Z'



### Populating batch source

Feast is agnostic to how the batch source is populated, as long as it complies to the Feature Table specification. Therefore, any existing ETL tools can be used for the purpose of data ingestion. Alternatively, you can also use Feast SDK to ingest a Panda Dataframe to the batch source.

In [10]:
import pandas as pd
import numpy as np
from datetime import datetime

In [11]:
def generate_entities():
    return np.random.choice(999999, size=100, replace=False)

In [12]:
def generate_trips(entities):
    df = pd.DataFrame(columns=["driver_id", "trips_today", "datetime", "created"])
    df['driver_id'] = entities
    df['trips_today'] = np.random.randint(0, 1000, size=100).astype(np.int32)
    df['datetime'] = pd.to_datetime(
            np.random.randint(
                datetime(2020, 10, 10).timestamp(),
                datetime(2020, 10, 20).timestamp(),
                size=100),
        unit="s"
    )
    df['created'] = pd.to_datetime(datetime.now())
    return df
    

In [13]:
def generate_stats(entities):
    df = pd.DataFrame(columns=["driver_id", "conv_rate", "acc_rate", "avg_daily_trips", "datetime", "created"])
    df['driver_id'] = entities
    df['conv_rate'] = np.random.random(size=100).astype(np.float32)
    df['acc_rate'] = np.random.random(size=100).astype(np.float32)
    df['avg_daily_trips'] = np.random.randint(0, 1000, size=100).astype(np.int32)
    df['datetime'] = pd.to_datetime(
            np.random.randint(
                datetime(2020, 10, 10).timestamp(),
                datetime(2020, 10, 20).timestamp(),
                size=100),
        unit="s"
    )
    df['created'] = pd.to_datetime(datetime.now())
    return df

In [14]:
entities = generate_entities()
stats_df = generate_stats(entities)
trips_df = generate_trips(entities)

In [15]:
client.ingest(driver_statistics, stats_df)
client.ingest(driver_trips, trips_df)

Removing temporary file(s)...
Data has been successfully ingested into FeatureTable batch source.
Removing temporary file(s)...
Data has been successfully ingested into FeatureTable batch source.


## Historical Retrieval For Training

Create a training dataset from offline feature tables

In [16]:
import gcsfs
from pyarrow.parquet import ParquetDataset
from urllib.parse import urlparse

In [17]:
entities_with_timestamp = pd.DataFrame(columns=['driver_id', 'event_timestamp'])
entities_with_timestamp['driver_id'] = np.random.choice(entities, 10, replace=False)
entities_with_timestamp['event_timestamp'] = pd.to_datetime(np.random.randint(
    datetime(2020, 10, 18).timestamp(),
    datetime(2020, 10, 20).timestamp(),
    size=10), unit='s')
entities_with_timestamp

,driver_id,event_timestamp
0,291265,2020-10-19 13:50:53
1,389275,2020-10-19 21:55:08
2,81792,2020-10-18 08:37:32
3,150997,2020-10-18 03:21:28
4,227660,2020-10-19 12:48:11
5,862125,2020-10-18 11:54:01
6,247351,2020-10-18 14:20:03
7,20501,2020-10-18 19:38:20
8,888946,2020-10-19 23:06:46
9,150771,2020-10-19 19:00:26


In [18]:
# get_historical_features will return immediately once the Spark job has been submitted succesfully.
job = client.get_historical_features(
    feature_refs=[
        "driver_statistics:avg_daily_trips",
        "driver_statistics:conv_rate",
        "driver_statistics:acc_rate",
        "driver_trips:trips_today"
    ], 
    entity_source=entities_with_timestamp
)

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [19]:
# get_output_file_uri will block until the Spark job is completed.
output_file_uri = job.get_output_file_uri()
print(output_file_uri)

gs://my-feast-playground-s-11-4744ea70/historical/d72a3c41-63b0-4c3a-81ce-708a82cf6d74


In [20]:
# Retrieve the remote training dataset
time.sleep(10)
parsed_uri = urlparse(output_file_uri)
fs = gcsfs.GCSFileSystem()
files = ["gs://" + path for path in fs.glob(output_file_uri + '/part-*')]
ds = ParquetDataset(files, filesystem=fs)
ds.read().to_pandas()
time.sleep(5)

The retrieved result can now be used for model training.

## Populating Online Storage with Batch Ingestion

In order to populate the online storage, we can use Feast SDK to start a Spark batch job which will extract the features from the batch source, then load the features to an online store.

In [21]:
job = client.start_offline_to_online_ingestion(
    driver_statistics,
    datetime(2020, 10, 10),
    datetime(2020, 10, 20)
)

In [22]:
# It will take some time before the Spark Job is completed
time.sleep(15)
job.get_status()

<SparkJobStatus.IN_PROGRESS: 1>

Once the job is completed, the SDK can be used to retrieve the result from the online store.

In [26]:
entities_sample = np.random.choice(entities, 10, replace=False)
entities_sample = [{"driver_id": e} for e in entities_sample]
entities_sample
time.sleep(15)


/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
time.sleep(15)

features = client.get_online_features(
    feature_refs=["driver_statistics:avg_daily_trips"],
    entity_rows=entities_sample).to_dict()
features


In [25]:
pd.DataFrame(features)

,driver_id,driver_statistics:avg_daily_trips
0,640707,None
1,453379,None
2,227660,None
3,438103,None
4,393016,None
5,861959,None
6,888946,None
7,542137,None
8,209409,None
9,829291,None
